## Pandas analysis

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [ ]:
# If you didn't download it yet, please get the relevant file now!
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ~/data/

1\. Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

2\. Find out the number of BX in a ORBIT (the value 'x').

3\. Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.

4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

6\. Create a new DataFrame that contains only the rows with HEAD=1.

7\. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

In [55]:
import pandas as pd
import numpy as np

path = "/home/sarte/data/data_000637.txt"
N = 500000000

data = pd.read_csv(path, nrows=N)
data.head(10)


HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
0     1     0          123  3869200167        2374        26
1     1     0          124  3869200167        2374        27
2     1     0           63  3869200167        2553        28
3     1     0           64  3869200167        2558        19
4     1     0           64  3869200167        2760        25
5     1     0           63  3869200167        2762         4
6     1     0           61  3869200167        2772        14
7     1     0          139  3869200167        2776         0
8     1     0           62  3869200167        2774        21
9     1     0           60  3869200167        2788         7

In [56]:
# Point 2
# data["BX_COUNTER"].value_counts()
X_MAX = max(data["BX_COUNTER"]) + 1
print("Il valore X cercato è", X_MAX)

Il valore X cercato è 3564


In [57]:
#point 3

max_orbit = np.max(data["ORBIT_CNT"])
max_bx = np.max(data[data["ORBIT_CNT"] == max_orbit]["BX_COUNTER"])
max_tdc = np.max(data[data["BX_COUNTER"] == max_bx]["TDC_MEAS"])

min_orbit = np.min(data["ORBIT_CNT"])
min_bx = np.min(data[data["ORBIT_CNT"] == min_orbit]["BX_COUNTER"])
min_tdc = np.min(data[data["BX_COUNTER"] == min_bx]["TDC_MEAS"])

max_ns = max_orbit * X_MAX * 25 +  max_bx * 25 + max_tdc * 25/30
min_ns = min_orbit * X_MAX * 25 +  min_bx * 25 + min_tdc * 25/30

total_s = (max_ns - min_ns) * 1e-9
print("Secondi totali:", total_s)



0.9804162991875001


In [66]:
#point 4

data["ABSOLUTE_TIME"] = data["ORBIT_CNT"] * X_MAX * 25 +  data["BX_COUNTER"] * 25 + data["TDC_MEAS"] * 25/30

#point 5

data["HEAD"] = np.random.randint(0, 2, size=len(data))
# data["HEAD"].mean()
data.head(15)

HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS  ABSOLUTE_TIME
0      1     0          123  3869200167        2374        26   3.447457e+14
1      1     0          124  3869200167        2374        27   3.447457e+14
2      0     0           63  3869200167        2553        28   3.447457e+14
3      1     0           64  3869200167        2558        19   3.447457e+14
4      0     0           64  3869200167        2760        25   3.447457e+14
5      0     0           63  3869200167        2762         4   3.447457e+14
6      0     0           61  3869200167        2772        14   3.447457e+14
7      1     0          139  3869200167        2776         0   3.447457e+14
8      0     0           62  3869200167        2774        21   3.447457e+14
9      0     0           60  3869200167        2788         7   3.447457e+14
10     1     1            7  3869200167        2785         4   3.447457e+14
11     0     0           64  3869200167        2786        19   3.447457e+14
12     0     1            6  3869200167        2792        18   3.447457e+14
13     1     0           36  3869200167        2791        23   3.447457e+14
14     1     0           56  3869200167        2789         3   3.447457e+14